## Problem 1

In Matlab or Python, implement a version of the TSQR that divides an input matrix up into four blocks of rows (using row sub-indexing) and computes the QR-factorisation in the way shown in the lectures on communication-avoiding factorisations.

TSQR defines a family of algorithms, in which the QR factorization of A is obtained by performing a sequence of QR factorizations until the lower trapezoidal part of A is annihilated and the final R factor is obtained. The QR factorizations are performed on block rows of A and on previously obtained R factors, stacked atop one another. We call the pattern followed during this sequence of QR factorizations a reduction tree.

### Sequential TSQR

Sequential TSQR uses a similar factorization process, but with a “flat tree” (a linear chain). We start with the same block row decomposition as with parallel TSQR,
but begin with a QR factorization of $A_0$, rather than of all the block rows:

$$
A =
\begin{pmatrix}
A_0 \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
Q_{00} R_{00} \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
$$

This is “stage 0” of the computation, hence the second subscript 0 of the $Q$ and $R$ factor. We then combine $R_{00}$ and $A_1$ using a QR factorization:

$$
\begin{pmatrix}
R_{00} \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
R_{00} \\
A_1 \\
\hline
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
Q_{01} R_{01} \\
\hline
A_2 \\
A_3
\end{pmatrix}
$$

We continue this process until we run out of $A_i$ factors. Here, the $A_i$ blocks are $m/P \times n$. If we were to compute all the above $Q$ factors explicitly as square matrices, which we do not, then $Q_{00}$ would be $m/P \times m/P$ and $Q_{0j}$ for $j > 0$ would be $2m/P \times 2m/P$ . The final R factor, as in the parallel case, would be $m \times n$ upper triangular (or $n \times n$ upper triangular in a “thin QR”).

In [ ]:
# Reusing code from Numerical Methods (MAP55631) Assignment 3
import numpy as np
from typing import Tuple


def mgs(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """
  Computes the Modified Grahm-Schmidt Algorithm.

  Args:
    A: an input matrix

  Returns:
    Tuple[np.ndarray, np.ndarray]: matrixes Q, R
      as the result of the factorization algorithm.
  """
  # Get shape from input matrix A
  m, n = A.shape

  # Create zeros matrix for Q with size m x n (same as A)
  # Q contains the orthonormal vectors that span the columns space of A
  Q = np.zeros((m, n))

  # Create zeros matrix R with size n x n.
  # R is an upper triangular matrix
  R = np.zeros((n, n))

  # Initialize V as copy of A
  V = A.copy()
  V = V.astype(np.float64)

  # Iterate over the column spaces of A
  for i in range(n):
    v_i = V[:, i]

    # Compute and store the norm of the vector
    r_ii = np.linalg.norm(v_i)
    R[i, i] = r_ii

    # Compute and store the orthonormal vector
    q_i = v_i / r_ii
    Q[:, i] = q_i

    # Iterate over the subsequent column spaces of A
    for j in range(i + 1, n):
      v_j = V[:, j]

      # Compute and store the values of R
      r_ij = np.dot(q_i, v_j)
      R[i, j] = r_ij

      # Update and store the orthogonal vector
      v_j = v_j - r_ij * q_i
      V[:, j] = v_j

  return Q, R

In [ ]:
# Sequential implementation of TSQR
p = 4                                     # 4 block rows
m = 4 * p                                 # So each block has 4 rows
n = 5                                     # So each block has 5 columns
scaling_factor = 100                      # Scales elements, otherwise [0, 1)
A = scaling_factor * np.random.rand(m,n)  # Matrix
print(f"A ({m} x {n}):\n{A}\n")

# Partioning A into p blocks
m = A.shape[0] // p
A_partitioned = [None] * p
for i in range(p):
  A_partitioned[i] = A[m * i: m * (i + 1), :]
  print(f"A_{i} ({m} x {n}):")
  print(f"{A_partitioned[i]}\n")

R = None
for i in range(p):
  if i == 0:
    # QR factorization of A_0
    _, R = mgs(A_partitioned[i])
    print(f"R_0{i} ({R.shape[0]} x {R.shape[1]}):\n{R}\n")
  else:
    # Combine R_i-1 and A_i using a QR factorization
    _, R = mgs(np.vstack((R, A_partitioned[i])))
    print(f"R_0{i} ({R.shape[0]} x {R.shape[1]}):\n{R}\n")

A (16 x 5):
[[34.1572165  21.72268007 65.54051431 76.64354738 44.643707  ]
 [ 1.41145446 32.05911413 32.23835764 50.54438836  5.97530075]
 [94.24281117 85.22069052 56.27136413 87.20651082 92.00762581]
 [69.2376024  88.12161375 75.5770929  20.74396316 54.72859737]
 [ 9.7568212  36.6964823  73.80374974 64.55011863 76.89632771]
 [82.74091577 76.78461728 54.61029379 66.67638665 67.70609209]
 [85.16715277 61.08335699 47.07965789 67.61358576 72.02214555]
 [52.8480907  28.57517341 84.78079193 10.45826031 36.08808044]
 [61.82156914 95.24151836  4.59728    37.49612165  7.06004991]
 [81.57417923 88.4728492  90.07348106  4.33301899 99.70415142]
 [53.94841514 41.46845309 96.26794828 25.58718201 85.0031129 ]
 [61.30285759 28.83154644 44.66706791 43.15726049 91.66239379]
 [47.14410896 71.94691231 40.56471179 83.49636391 80.86870032]
 [84.33560381 91.71584817 82.29106198 48.65553016 10.2297816 ]
 [60.44621231 80.18153101 35.45623732 82.17832829 78.71792432]
 [97.26961153 51.87399115 46.34693454 38.19

References:
- [1] Jammes Demmel, Laura Grigori, Mark Hoemmen, and Julien Langou. Implementing Communicaiton-Optimal Parallel And Sequential QR Factorizations. https://arxiv.abs/pdf/0809.2407, 2008. arXiv:0809.2407 [math.NA]